In [1]:
from tensorflow.keras.layers import Dense, Input, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE=[224,224]
train_path='train/'
test_path='test/'

In [3]:
resnet50 = ResNet50(input_shape=IMAGE_SIZE +[3], weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
resnet50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
for layer in resnet50.layers:
    layer.trainable=False

In [6]:
folders = glob('train/*')
len(folders)

3

In [7]:
x = Flatten()(resnet50.output)
predictions = Dense(len(folders), activation='softmax')(x)

In [8]:
model = Model(inputs=resnet50.input, outputs=predictions)

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [11]:
train_data_gen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_data_gen = ImageDataGenerator(rescale=1.0/255)

In [12]:
training_set = train_data_gen.flow_from_directory(directory='train',target_size=(224,224),batch_size=32, class_mode='categorical')
test_set = test_data_gen.flow_from_directory(directory='test',target_size=(224,224),batch_size=32, class_mode='categorical')

Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.


In [13]:
r = model.fit_generator(training_set, validation_data=test_set,epochs=5, steps_per_epoch=len(training_set),
                        validation_steps=len(test_set))

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
2/2 [==============================] - 39s 19s/step - loss: 5.4735 - acc: 0.3125 - val_loss: 2.9495 - val_acc: 0.3276
Epoch 2/5
2/2 [==============================] - 34s 17s/step - loss: 4.5095 - acc: 0.6250 - val_loss: 9.0855 - val_acc: 0.3276
Epoch 3/5
2/2 [==============================] - 34s 17s/step - loss: 2.5868 - acc: 0.8125 - val_loss: 10.6356 - val_acc: 0.3276
Epoch 4/5
2/2 [==============================] - 35s 18s/step - loss: 1.6494 - acc: 0.8125 - val_loss: 10.8100 - val_acc: 0.3276
Epoch 5/5
2/2 [==============================] - 35s 17s/step - loss: 0.9585 - acc: 0.9219 - val_loss: 10.6356 - val_acc: 0.3276
